### 5. Part-of-Speech разметка, NER, извлечение отношений

Задание 1. Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы, сделать выводы  


Задание 2. Проверить, насколько хорошо работает NER
Данные брать из Index of /pub/named_entities
проверить NER из nltk/spacy/deeppavlov.
написать свой NER, попробовать разные подходы.
передаём в сетку токен и его соседей.
передаём в сетку только токен.
свой вариант.
сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.



#### Загрузим необходимые библиотеки и данные.

In [2]:
!pip install pyconll

In [ ]:
!pip install wget

In [3]:
!wget -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2023-10-05 14:00:12--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736581 (39M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train.conllu’

ru_syntagrus-ud-tra 100%[===================>]  38.85M   256MB/s    in 0.2s    

2023-10-05 14:00:12 (256 MB/s) - ‘ru_syntagrus-ud-train.conllu’ saved [40736581/40736581]

--2023-10-05 14:00:12--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108

In [4]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import pyconll
import nltk
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger, RegexpTagger

import warnings
warnings.filterwarnings("ignore")

In [5]:
data_train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
data_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [6]:
fdata_train = []
for sent in data_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])

fdata_test = []
for sent in data_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])

fdata_sent_test = []
for sent in data_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [7]:
len(fdata_train), len(fdata_test), len(fdata_sent_test)

(24516, 8906, 8906)

In [8]:
fdata_train[:2]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')]]

#### Проверим работу всех теггеров

In [9]:
default_tagger = nltk.DefaultTagger('NN')
default_acc = default_tagger.evaluate(fdata_test)

unigram_tagger = UnigramTagger(fdata_train)
unigram_acc = unigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train)
bigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train)
trigram_acc = trigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_unigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_bigram_unigram_acc = trigram_tagger.evaluate(fdata_test)

print(f'Accuracy:\nDefault Tagger: {round(default_acc, 3)},\nUnigram Tagger: {round(unigram_acc, 3)},\nBigram Tagger: {round(bigram_acc, 5)},\n'
      f'Trigram Tagger: {round(trigram_acc, 3)},\nBigram and Unigram Tagger: {round(bigram_unigram_acc, 5)},\n'
      f'Trigram, Bigram and Unigram Tagger: {round(trigram_bigram_unigram_acc, 5)},\n')

Accuracy:
Default Tagger: 0.0,
Unigram Tagger: 0.824,
Bigram Tagger: 0.60939,
Trigram Tagger: 0.178,
Bigram and Unigram Tagger: 0.82928,
Trigram, Bigram and Unigram Tagger: 0.82914,



Объединим работу всех теггеров с помощью функци

In [10]:
def union_taggers(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN')
tag = union_taggers(fdata_train,
                     [UnigramTagger, BigramTagger, TrigramTagger],
                     backoff = backoff)

tag.evaluate(fdata_test)

0.827905462595221

#### Пишем кастомный tagger

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [12]:
#создадим списки слов и списки POS-разметки

train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])

test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(' ' if tok[0] is None else tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])


In [13]:
train_tok[:10], train_label[:10]

(['Анкета',
  '.',
  'Начальник',
  'областного',
  'управления',
  'связи',
  'Семен',
  'Еремеевич',
  'был',
  'человек'],
 ['NOUN',
  'PUNCT',
  'NOUN',
  'ADJ',
  'NOUN',
  'NOUN',
  'PROPN',
  'PROPN',
  'AUX',
  'NOUN'])

In [14]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
train_enc_labels

array([ 7, 13,  7, ...,  1, 11, 13])

In [20]:
test_enc_labels = le.transform(test_label)
test_enc_labels

array([ 7, 13,  1, ...,  0,  7, 13])

In [15]:
#получившиеся классы
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

Проведем веткоризацию и обучим модель логистической регрессии

In [21]:
%time

vectorizers = [CountVectorizer(ngram_range=(1, 5), analyzer='char'),
               TfidfVectorizer(ngram_range=(1, 5), analyzer='char'),
               HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=1000)]
vectorizers_word = [CountVectorizer(ngram_range=(1, 5), analyzer='word'),
               TfidfVectorizer(ngram_range=(1, 5), analyzer='word'),
               HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=1000)]
n_features = [2000, 3000, 5000]
vectorizers_hash = [HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok)

    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels, pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels, pred)
    accuracy_scores.append(acc)

    print(vectorizer)
    print(classification_report(test_enc_labels, pred, target_names=le.classes_))

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 27.9 µs
CountVectorizer(analyzer='char', ngram_range=(1, 5))
              precision    recall  f1-score   support

         ADJ       0.94      0.92      0.93     15103
         ADP       0.98      1.00      0.99     13717
         ADV       0.91      0.93      0.92      7783
         AUX       0.82      0.96      0.88      1390
       CCONJ       0.89      0.97      0.93      5672
         DET       0.83      0.79      0.81      4265
        INTJ       0.39      0.29      0.33        24
        NOUN       0.94      0.97      0.96     36238
      NO_TAG       1.00      0.77      0.87       265
         NUM       0.86      0.89      0.88      1734
        PART       0.95      0.77      0.85      5125
        PRON       0.90      0.84      0.87      7444
       PROPN       0.84      0.66      0.73      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.75      0.97      0.85      2865
         SYM  

Соберем данные обучения в таблицу

In [22]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [26]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'f1_score': f1_scores, 'Accuracy': accuracy_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(1, 5))",0.938119,0.939469
1,"TfidfVectorizer(analyzer='char', ngram_range=(1, 5))",0.926624,0.929012
8,"HashingVectorizer(analyzer='char', n_features=5000, ngram_range=(1, 5))",0.909308,0.912540
7,"HashingVectorizer(analyzer='char', n_features=3000, ngram_range=(1, 5))",0.898547,0.902070
6,"HashingVectorizer(analyzer='char', n_features=2000, ngram_range=(1, 5))",0.892840,0.896243
2,"HashingVectorizer(analyzer='char', n_features=1000, ngram_range=(1, 5))",0.866679,0.870441
4,"TfidfVectorizer(ngram_range=(1, 5))",0.654096,0.643336
3,"CountVectorizer(ngram_range=(1, 5))",0.648576,0.637971
11,"HashingVectorizer(n_features=5000, ngram_range=(1, 5))",0.581928,0.598294
10,"HashingVectorizer(n_features=3000, ngram_range=(1, 5))",0.567710,0.592669


Вывод: векторайзеры на N-граммах отработали лучше, символьные показали наилучший результат, чем словарные

### Задание 2. Проверить, насколько хорошо работает NER

Установим и имоптируем библиотеки

In [27]:
!pip install corus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 1.9 MB/s eta 0:00:00


In [28]:
!pip install razdel


In [29]:
!pip install -U spacy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.1 which is incompatible.


In [30]:
!python -m spacy info

2023-10-05 14:29:39.842366: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-05 14:29:43.907384: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)

============================== Info about spaCy ==============================

spaCy version    3.7.1                      

In [31]:
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
import pyconll
import corus
from corus import load_ne5
import re
import spacy
from spacy import displacy


In [32]:
#подтянем словари
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [33]:
nltk.help.upenn_tagset('RB')
nltk.help.upenn_tagset('NN')
nltk.help.upenn_tagset('VB')

RB: adverb
    occasionally unabatingly maddeningly adventurously professedly
    stirringly prominently technologically magisterially predominately
    swiftly fiscally pitilessly ...
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
VB: verb, base form
    ask assemble assess assign assume atone attention avoid bake balkanize
    bank begin behold believe bend benefit bevel beware bless boil bomb
    boost brace break bring broil brush build ...


In [34]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2023-10-05 14:32:34--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  2.28MB/s    in 0.8s    

2023-10-05 14:32:36 (2.28 MB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [35]:
!unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

Теперь посмотим на данные

In [36]:
records = load_ne5('Collection5/')

In [37]:
document = next(records)
text = document.text

text = re.sub('\r\n\r\n',' ',text)
text = re.sub('\r\n',' ',text)
text

'В Финляндии сменился премьер-министр Парламент Финляндии в среду, 22 июня, проголосовал по кандидатуре премьер-министра страны. Как сообщает финская телерадиовещательная компания YLE, им стал бывший министр финансов Юрки Катайнен (Jyrki Katainen). Глава правительства был избран спустя более чем два месяца после парламентских выборов. Как сообщает Agence France-Presse, для Финляндии этот срок является рекордным. Победителем на выборах, прошедших 17 апреля, стала Коалиционная партия, во главе которой стоит Катайнен. Вместе с ней в коалицию также войдут Социал-демократическая партия, Союз левых сил, "Зеленые", Шведская народная партия, а также Христианские демократы. В то же время националистическая партия "Истинные финны" от участия в правительстве отказалась. В среду кандидатуру Катайнена на посту премьер-министр утвердит президент Финляндии Тарья Халонен. '

In [40]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
document.text = text
# Предварительная очистка статей в словаре
nltk.pos_tag(nltk.word_tokenize(text))

[('В', 'JJ'),
 ('Финляндии', 'NNP'),
 ('сменился', 'NNP'),
 ('премьер-министр', 'JJ'),
 ('Парламент', 'NNP'),
 ('Финляндии', 'NNP'),
 ('в', 'NNP'),
 ('среду', 'NNP'),
 (',', ','),
 ('22', 'CD'),
 ('июня', 'NN'),
 (',', ','),
 ('проголосовал', 'NNP'),
 ('по', 'NNP'),
 ('кандидатуре', 'NNP'),
 ('премьер-министра', 'JJ'),
 ('страны', 'NNP'),
 ('.', '.'),
 ('Как', 'VB'),
 ('сообщает', 'JJ'),
 ('финская', 'NNP'),
 ('телерадиовещательная', 'NNP'),
 ('компания', 'NNP'),
 ('YLE', 'NNP'),
 (',', ','),
 ('им', 'NNP'),
 ('стал', 'NNP'),
 ('бывший', 'NNP'),
 ('министр', 'NNP'),
 ('финансов', 'NNP'),
 ('Юрки', 'NNP'),
 ('Катайнен', 'NNP'),
 ('(', '('),
 ('Jyrki', 'NNP'),
 ('Katainen', 'NNP'),
 (')', ')'),
 ('.', '.'),
 ('Глава', 'JJ'),
 ('правительства', 'JJ'),
 ('был', 'NN'),
 ('избран', 'NNP'),
 ('спустя', 'NNP'),
 ('более', 'NNP'),
 ('чем', 'NNP'),
 ('два', 'NNP'),
 ('месяца', 'NNP'),
 ('после', 'NNP'),
 ('парламентских', 'NNP'),
 ('выборов', 'NNP'),
 ('.', '.'),
 ('Как', 'VB'),
 ('сообщает', 'J

In [42]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) if hasattr(chunk, 'label') }

{('Jyrki Katainen', 'PERSON'),
 ('Парламент Финляндии', 'PERSON'),
 ('Союз', 'PERSON'),
 ('Финляндии', 'PERSON'),
 ('Шведская', 'PERSON')}

In [43]:
document.spans

[Ne5Span(
     index='T1',
     type='GEOPOLIT',
     start=2,
     stop=11,
     text='Финляндии'
 ),
 Ne5Span(
     index='T2',
     type='GEOPOLIT',
     start=50,
     stop=59,
     text='Финляндии'
 ),
 Ne5Span(
     index='T3',
     type='MEDIA',
     start=182,
     stop=185,
     text='YLE'
 ),
 Ne5Span(
     index='T4',
     type='PER',
     start=219,
     stop=249,
     text='Юрки Катайнен (Jyrki Katainen)'
 ),
 Ne5Span(
     index='T5',
     type='MEDIA',
     start=355,
     stop=375,
     text='Agence France-Presse'
 ),
 Ne5Span(
     index='T6',
     type='GEOPOLIT',
     start=381,
     stop=390,
     text='Финляндии'
 ),
 Ne5Span(
     index='T7',
     type='ORG',
     start=475,
     stop=494,
     text='Коалиционная партия'
 ),
 Ne5Span(
     index='T8',
     type='PER',
     start=519,
     stop=527,
     text='Катайнен'
 ),
 Ne5Span(
     index='T9',
     type='ORG',
     start=566,
     stop=595,
     text='Социал-демократическая партия'
 ),
 Ne5Span(
     index='

Поработаем со SpaCy

In [44]:
!python -m spacy download ru_core_news_sm

2023-10-05 14:40:55.436191: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 55.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [45]:
import spacy
from spacy import displacy
import ru_core_news_sm
from spacy.lang.ru.examples import sentences
from spacy.lang.ru import Russian

Загрузим данные

In [46]:
nlp = spacy.load("ru_core_news_sm")

In [47]:
news_bb = text
article = nlp(news_bb)

Подсветим теги

In [48]:
displacy.render(article, jupyter=True, style='ent')

In [49]:
#Покажем списком токены и части речи
for token in article:
    print(token.text, token.pos_, token.dep_)

В ADP case
Финляндии PROPN obl
сменился VERB ROOT
премьер NOUN nsubj
- NOUN nsubj
министр NOUN nsubj
Парламент PROPN appos
Финляндии PROPN nmod
в ADP case
среду NOUN obl
, PUNCT punct
22 ADJ nmod
июня NOUN flat
, PUNCT punct
проголосовал VERB conj
по ADP case
кандидатуре NOUN obl
премьер NOUN nmod
- NOUN nmod
министра NOUN nmod
страны NOUN nmod
. PUNCT punct
Как SCONJ mark
сообщает VERB parataxis
финская ADJ amod
телерадиовещательная ADJ amod
компания NOUN nsubj
YLE PROPN appos
, PUNCT punct
им PRON xcomp
стал VERB ROOT
бывший ADJ amod
министр NOUN nsubj
финансов NOUN nmod
Юрки PROPN appos
Катайнен PROPN flat:name
( PUNCT punct
Jyrki PROPN appos
Katainen PROPN flat:name
) PUNCT punct
. PUNCT punct
Глава NOUN nsubj:pass
правительства NOUN nmod
был AUX aux:pass
избран VERB ROOT
спустя ADP case
более ADV case
чем SCONJ fixed
два NUM nummod:gov
месяца NOUN obl
после ADP case
парламентских ADJ amod
выборов NOUN nmod
. PUNCT punct
Как SCONJ mark
сообщает VERB parataxis
Agence X nsubj
France 

Написать свой NER

In [52]:
from sklearn.metrics import confusion_matrix, classification_report,f1_score, accuracy_score

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from razdel import tokenize
from corus import load_ne5

In [53]:
def get_classification_report(y_test_true, y_test_pred):
    print(classification_report(y_test_true, y_test_pred))

    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [54]:
#Возьмем корпус текстов
records = load_ne5('Collection5/')
next(records)

Ne5Markup(
    id='315',
    text='В Финляндии сменился премьер-министр\r\n\r\nПарламент Финляндии в среду, 22 июня, проголосовал по кандидатуре премьер-министра страны. Как сообщает финская телерадиовещательная компания YLE, им стал бывший министр финансов Юрки Катайнен (Jyrki Katainen).\r\n\r\nГлава правительства был избран спустя более чем два месяца после парламентских выборов. Как сообщает Agence France-Presse, для Финляндии этот срок является рекордным.\r\n\r\nПобедителем на выборах, прошедших 17 апреля, стала Коалиционная партия, во главе которой стоит Катайнен. Вместе с ней в коалицию также войдут Социал-демократическая партия, Союз левых сил, "Зеленые", Шведская народная партия, а также Христианские демократы. В то же время националистическая партия "Истинные финны" от участия в правительстве отказалась.\r\n\r\nВ среду кандидатуру Катайнена на посту премьер-министр утвердит президент Финляндии Тарья Халонен. ',
    spans=[Ne5Span(
         index='T1',
         type='GEOPOLIT',

In [55]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [58]:
#Сделаем новый датафрейм
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [59]:
df_words['tag'].value_counts()

OUT         219110
PER          21190
ORG          13636
LOC           4568
GEOPOLIT      4352
MEDIA         2479
Name: tag, dtype: int64

In [60]:
df_words.head()

,word,tag
0,В,PER
1,.,PER
2,Путин,PER
3,освободил,OUT
4,от,OUT


In [61]:
#Распределим на трейн и тест
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

In [62]:
#Зададим целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [63]:
encoder.classes_  #всего 6 классов

array(['GEOPOLIT', 'LOC', 'MEDIA', 'ORG', 'OUT', 'PER'], dtype=object)

In [64]:
train_x.apply(len).max(axis=0)

55

In [65]:
valid_x

245016             ранее
264665             Центр
78107            премьер
246937             равно
68356      представитель
               ...      
121106          позволит
41877     Исполнительный
126486            Замана
10461             газета
166858                 в
Name: word, Length: 66334, dtype: object

In [66]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [67]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [69]:
def custom_net(input_data):

    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
                            standardize=custom_net,
                            max_tokens=vocab_size,
                            output_mode='int',
                            output_sequence_length=seq_len)


text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [70]:
len(vectorize_layer.get_vocabulary())

29792

In [71]:
#эмбединги
embedding_dim = 32

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax') # [OUT, PER, ORG, LOC, GEOPOLIT, MEDIA]

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)

        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)

        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [72]:
mmodel = modelNER()

Компилируем и обучаем модель

In [73]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [74]:
mmodel.fit( train_data,
            validation_data=valid_data,
            epochs=3)

Epoch 1/3
12438/12438 [==============================] - 213s 16ms/step - loss: 0.3079 - accuracy: 0.9098 - val_loss: 0.2221 - val_accuracy: 0.9346
Epoch 2/3
12438/12438 [==============================] - 198s 16ms/step - loss: 0.1317 - accuracy: 0.9616 - val_loss: 0.3430 - val_accuracy: 0.8921
Epoch 3/3
12438/12438 [==============================] - 177s 14ms/step - loss: 0.1120 - accuracy: 0.9655 - val_loss: 0.2920 - val_accuracy: 0.8928


In [75]:
pred_y = mmodel.predict(valid_x)
y_pred_classes = np.argmax(pred_y,axis=1)
#предсказание модели

2073/2073 [==============================] - 5s 2ms/step


In [76]:
f1_score = f1_score(valid_y, y_pred_classes, average= "weighted")
f1_score

0.8996003324554235

In [77]:
print(f"Classes: {encoder.classes_}\r\n")

get_classification_report(valid_y, y_pred_classes)

Classes: ['GEOPOLIT' 'LOC' 'MEDIA' 'ORG' 'OUT' 'PER']

              precision    recall  f1-score   support

           0       0.90      0.89      0.89      1109
           1       0.87      0.76      0.81      1127
           2       0.92      0.77      0.84       627
           3       0.87      0.55      0.67      3455
           4       0.97      0.92      0.94     54660
           5       0.50      0.88      0.63      5356

    accuracy                           0.89     66334
   macro avg       0.84      0.80      0.80     66334
weighted avg       0.92      0.89      0.90     66334

CONFUSION MATRIX

col_0    0    1    2     3      4     5
row_0                                  
0      985    7    1    61      9    46
1       29  858    0    22     43   175
2        5    7  485     8     67    55
3       64   38   26  1910   1010   407
4       13   75   14   200  50290  4068
5        0    2    0     5    651  4698


In [78]:
def custom_net(input_data):
    # обучим сеть на биграммах и триграммах
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
                            standardize=custom_net,
                            max_tokens=vocab_size,
                            output_mode='int',
                            ngrams=(1, 3),
                            output_sequence_length=seq_len)


text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [79]:
mmodel = modelNER()

Компилируем и обучаем модель

In [80]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [81]:
mmodel.fit( train_data,
            validation_data=valid_data,
            epochs=3)

Epoch 1/3
12438/12438 [==============================] - 201s 16ms/step - loss: 0.3058 - accuracy: 0.9103 - val_loss: 0.2232 - val_accuracy: 0.9359
Epoch 2/3
12438/12438 [==============================] - 204s 16ms/step - loss: 0.1308 - accuracy: 0.9617 - val_loss: 0.2377 - val_accuracy: 0.9387
Epoch 3/3
12438/12438 [==============================] - 208s 17ms/step - loss: 0.1126 - accuracy: 0.9654 - val_loss: 0.2670 - val_accuracy: 0.9396


In [82]:
pred_y = mmodel.predict(valid_x)
y_pred_classes = np.argmax(pred_y,axis=1)
#предсказание модели

2073/2073 [==============================] - 8s 4ms/step


In [84]:
print(f"Classes: {encoder.classes_}\r\n")

get_classification_report(valid_y, y_pred_classes)

Classes: ['GEOPOLIT' 'LOC' 'MEDIA' 'ORG' 'OUT' 'PER']

              precision    recall  f1-score   support

           0       0.90      0.88      0.89      1109
           1       0.86      0.76      0.81      1127
           2       0.93      0.78      0.84       627
           3       0.88      0.54      0.67      3455
           4       0.94      0.99      0.97     54660
           5       0.98      0.71      0.82      5356

    accuracy                           0.94     66334
   macro avg       0.91      0.78      0.83     66334
weighted avg       0.94      0.94      0.93     66334

CONFUSION MATRIX

col_0    0    1    2     3      4     5
row_0                                  
0      981    7    0    58     62     1
1       28  857    0    27    214     1
2        4    7  486     6    124     0
3       65   39   25  1877   1433    16
4        9   79   14   168  54318    72
5        0    2    0     5   1543  3806



На биграммах и триграммах метрики показали лучший результат